In [1]:
from qiskit import *
import random
from math import pi
import matplotlib.pyplot as plt


In [2]:
# Plot results
def show_results(D):
    # D is a dictionary with classical bits as keys and count as value
    # example: D = {'000': 497, '001': 527}
    plt.bar(range(len(D)), list(D.values()), align='center')
    plt.xticks(range(len(D)), list(D.keys()))
    plt.show()

# Execute circuit, display a histogram of the results
def execute_locally(qc, draw_circuit=False):
    # Compile and run the Quantum circuit on a simulator backend
    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = execute(qc, backend_sim, shots=1000)
    result_sim = job_sim.result()
    result_counts = result_sim.get_counts(qc)
    # Print the results
    print("simulation: \n\n", result_counts)
    show_results(result_counts)



In [3]:
def QFTransform(circuit,qr,swap=False):
    i=qr.size
    k=1
    
    for j in reversed(range(i)):
        circuit.h(qr[j])

        for l in reversed(range(j)):
            rot = pi / 2 ** k
            circuit.cu1(rot,qr[l],qr[j])
            k+=1
        k=1
        circuit.barrier()
    
    if swap and qr.size > 1:
        i=0
        j=qr.size-1
        while(i<j):
            circuit.swap(qr[i],qr[j])
            i+=1
            j-=1

def iQFTransform(circuit,qr,swap=False):
    for i in range(qr.size):
        k=i
        for l in range(i):
            rot = -(pi / 2 ** k)
            circuit.cu1(rot,qr[l],qr[i])
            k-=1
        circuit.h(qr[i])
        
        circuit.barrier()
    
    if swap and qr.size > 1:
        i=0
        j=qr.size-1
        while(i<j):
            circuit.swap(qr[i],qr[j])
            i+=1
            j-=1


 we have a weight vector -> W = {w1,w2,...,w6} and input vector X={x1,x2}
 The output of each hidden layer will serve as input of the output layer.
 For each layer we need feedforward operation to calculate $\sum_{n=1}^{2} W_{n}^{T} X_{n}$. After the summation we will consider a step function to make the activation function.
 FeedForward can be done in a quantum fashion by writing $\sum_{n=1}^{2} W_{n}^{T} X_{n}$ in the phase of a quantum register and then using quantum phase estimation to retrieve the approximate value of the summation into the auxiliary register -> $iQFT_{x}(U^{iW}( \ket x \otimes \ket 0))$ -> $\ket x \otimes \ket{W^{T} x}$
 <br>Next , in order to feed the output of the perceptron as an input to another perceptron we need to perform an activation on the auxiliary register. I'll use the step function because is simply measuring the first qubit of the auxiliary register and thats what we need for this case.<br>
 After the feedforward we have the output $\overline y$. We need to obtain the output for each training data in order to compute de loss function $MSE = \sum_{n=1}^{4}(\overline y - y)^{2}$

In [5]:
w=6
inputReg = QuantumRegister(2)
outputReg = QuantumRegister(1)
weightReg = QuantumRegister(w)
randWeight = [random.randint(0,1) for x in range(6)]

qc = QuantumCircuit(inputReg,outputReg,weightReg)

def xorStatePreparation(qc,inputReg,weightReg,randWeight,outputReg):
    qc.h(inputReg)
    qc.x(inputReg[0])
    qc.ccx(inputReg[0],inputReg[1],outputReg[0])
    qc.x(inputReg[0])
    qc.x(inputReg[1])
    qc.ccx(inputReg[0],inputReg[1],outputReg[0])
    qc.x(inputReg[1])
    for i in range(w):
        if randWeight[i] == 1:
            qc.x(weightReg[i])
    

 Create a class perceptron for easy and general use.
 easy to construct the xor neural network
 perceptron suggested by Maria Schuld et.al using the principle of quantum phase estimation (QPE)
 We begin with three concrete registers $\ket{x} \otimes \ket w \otimes \ket 0$
 Then we have an oracle that encodes $\sum_{n=1}^{2} W_{n}^{T} X_{n}$ into the phase of input register
 QPE is then able to retrieve the weighted sum approximately (greater than 90%) into the ancilla register
 One way of obtaining the output of the neuron is doing a step activation function on the ancilla register
 wich is just a measure on the most significant qubit of the ancilla register.

In [6]:
class qPerceptron:
    def __init__(self,qc,inputReg,weightReg):
        self.qc = qc
        self.inputReg = inputReg
        self.weightReg = weightReg
        self.ancillaReg = QuantumRegister(len(self.inputReg))
        self.qc.add_register(self.ancillaReg)

    def feedForward(self):
        return self.qPhaseEstimation()
    
    def qPhaseEstimation(self):
        self.qc.h(self.ancillaReg)
        self.qOracle(self.qc,self.inputReg,self.weightReg,self.ancillaReg)
        iQFTransform(qc,self.ancillaReg,swap=False)
        return self.ancillaReg[1]

    def qOracle(self,qc,inputReg,weightReg,ancillaReg):
        for i in range(ancillaReg.size):
            for j in reversed(range(i+1)):
                for k in reversed(range(len(weightReg))):
                    qc.h(inputReg[k])
                    qc.ccx(ancillaReg[i],weightReg[k],inputReg[k])
                    qc.h(inputReg[k])
    
    def setInputReg(self,inputReg):
        self.inputReg = inputReg
    


 xor-nn will be composed of an hidden layer with two perceptrons that constitutes the input layer. it will have an output layer with 1 perceptron
 feefforward operation

In [7]:
percep1 = qPerceptron(qc,inputReg,[weightReg[0],weightReg[1]])
out1 = percep1.feedForward()
percep2 = qPerceptron(qc,inputReg,[weightReg[2],weightReg[3]])
out2 = percep2.feedForward()
percep3 = qPerceptron(qc,[out1,out2],[weightReg[4],weightReg[5]])
out3 = percep3.feedForward()



 After the feedforward operation we want to know how bad/good are our predictions compared to the real value. This is done by computing
 the mean squared error function (MSE) as said above. Now, given that our input training data are given in superposition to the perceptrons,
 we are training in parallel and so , we only need to do MSE between out3 register and the output register in qc, and in parallel we will be
 computing the MSE for every training data points.
 For computing the $MSE = \sum_{n=1}^{4}(\overline y - y)^{2}$ we will need to do some quantum arithmetic to obtain the result in a quantum register
 and then backpropagate the error to the weight parameters
 Given that we need to do a subtraction and the registers will be of just one qubit , then we only need to perform xor operations with use of an
 extra ancilla qubit to store the error.

 Now that the xor-nn is constructed, lets construct a class called deepNN that will be responsible for training and testing the network after the
 training session. We need to specify the # of epochs that we want to train the nertwork.

 The purpose of Quantum BackPropagation procedure (QFB) is the same as in the classical analogue. We want to perform backpropagation of the gradient of the Loss Function -> gradient descent, because in this way we are walking towards the minimum of the loss function , so , we are walking towards the minimum error between our predictions and the real outputs. <br> QFB is based on the principal of the phase kickback. We gonna exponentiate the value of the loss function $L(\theta)$ into the parameters -> $e^{-i\eta L(\theta)}$ , with $\eta$ being the learning rate. Then, performing the Fourier Transform on these we shift the registers into the momentum space. Performing $e^{-i \gamma \theta ^{2}}$ on the parameters, we shift the momentum of these accordingly with the gradient of the loss function. -> $\theta = \theta - \eta \nabla(L(\theta))$ <br>
 We have 2 methods of making QFB: <br>
 <li>Performing one round of feedforward + backprop is called an epoch. Reverting the effect of the Fourier Transform in the parameters and procede to another epoch -> <b> Quantum Dynamical Descent (QDD) </b></li>
 <li> Measuring the parameters , performing an unitary to initiate the new parameters and only then procede to another epoch -> <b> Momentum Measurement Gradient Descent (MoMGrad) </b>
 Tipically $\eta$ in classical machine Learning is used to be $\eta$ = 0.1<br>
 Starting with large kinetic rate $\gamma >> \eta$ and with the increasinng of the #epochs change to $\gamma << \eta$ helps us to achieve the minimum of the function.
 Given that we are trying to work strictly with quantum data we will be using QDD

In [8]:

class deepNN:
    def __init__(self,qc,weightReg,perceptrons):
        self.qc = qc
        self.perceptrons = perceptrons
        self.weights = weightReg
        self.a = QuantumRegister(1)
        self.qc.add_register(self.a)

    def MSELossFunction(self,outputReg,out):
        self.qc.x(out)
        self.qc.ccx(out,outputReg,self.a[0])
        self.qc.x(out)
        self.qc.x(outputReg)
        self.qc.ccx(out,outputReg,self.a[0])
        self.qc.z(self.a[0])
        self.qc.x(outputReg)
        return self.a[0]
    
    def qDD(self,error,learnRate,kineticTerm):
        for w in self.weights:
            self.qc.cu1(learnRate,error,w)
        QFTransform(self.qc,self.weights,swap=False)
        for w in self.weights:
            self.qc.u1(kineticTerm,w)
        iQFTransform(self.qc,self.weights,swap=False)
        

    def train(self,outputReg,epochs,learnRate,kineticTerm):
        lr = learnRate
        kt = kineticTerm
        self.resetAncillas()
        for e in range(epochs):
            #routine for cleaning the ancilla registers of all perceptrons before initiate another epoch
            
            for p in self.perceptrons:
                out=p.feedForward()
            error = self.MSELossFunction(outputReg,out)
            self.qDD(error,lr,kt)
            self.qc.reset(self.a)
            self.resetAncillas()
            #update of learnRate and kineticTerm 
            lr = lr*2
            kt = kt/2

    def test(self,outputReg,cr):
        for p in self.perceptrons:
            out = p.feedForward()
        
        i=0
        for p in self.perceptrons[0].inputReg:
            self.qc.measure(p,cr[i])
            i+=1
        self.qc.measure(outputReg,cr[i])
        i+=1
        self.qc.measure(out,cr[i])

    def resetAncillas(self):
        for p in self.perceptrons:
            self.qc.reset(p.ancillaReg)


In [ ]:

xorNN = deepNN(qc,weightReg,[percep1,percep2,percep3])
#outputReg[0] , epochs , learnRate=0.1 , kineticTerm = 10 
epochs = 1
learnRate = 0.1
kineticTerm = 10

xorNN.train(outputReg[0],epochs,learnRate,kineticTerm)

newInput = QuantumRegister(2)
newOutput = QuantumRegister(1)
cr = ClassicalRegister(16)
qc.add_register(newInput,newOutput,cr)

testDatai = [(0,0),(0,1),(1,0),(1,1)]
testDatao = [0,1,1,0]


j=0
for i in range(1):
    if testDatai[i][0] == 1:
        qc.x(newInput[0])
    if testDatai[i][1] == 1:
        qc.x(newInput[1])
    if testDatao[i] == 1:
        qc.x(newOutput[0])
    
    percep1.setInputReg(newInput)

    xorNN.test(newOutput[0],[cr[j],cr[j+1],cr[j+2],cr[j+3]])
    j+=4
    xorNN.resetAncillas()
    qc.reset(newInput)
    qc.reset(newOutput)

execute_locally(qc,draw_circuit=False)

